<a href="https://colab.research.google.com/github/narcisocabraljr/Tech_Challenge_1/blob/main/Tech_Challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Começamos importando algumas bibliotecas básicas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np